In [76]:
from http.server import BaseHTTPRequestHandler, HTTPServer
from urllib.parse import urlparse, parse_qs
import sqlite3
import os
from datetime import datetime
import pytz
import hashlib
import random

Class tree:
-- GlobalOperations
--- Sqlite
---- Vibrateserver

In [145]:
class GlobalOperations:
    def __init__(self):
        self.tz = pytz.timezone('Asia/Jakarta')
    
    def datetime_now(self):
        return datetime.now(self.tz)
    
    def gethash(self, string):
        '''sha256'''
        # sanity check
        if not isinstance(string, str):
            string = str(string)            
        return hashlib.sha256(bytes(string, 'utf-8')).hexdigest()

class Sqlite(GlobalOperations):
    def __init__(self):
        self.conn = None
        self.table = None
        
    def set_table(self, table):
        self.table = table
        
    def create_connection(self, db_path):
        try:
            self.conn = sqlite3.connect(db_path)
        except Error as e:
            print(f'Failed to create connection. {e}')
    
    def insert_value(self, cols, values):
        # Sanity check
        if self.conn != None & self.table != None:
            if not all(isinstance(i, list) for i in [cols, values]):
                cols_insert = ','.join(cols)
                data_placeholder = ['?' for _ in cols]
                data_placeholder_insert = ','.join(data_placeholder)
                values_insert = f'{self.datetime_now().timestamp()}' + ','.join(values)
                sql = f'''insert into {self.table} (datetime, {cols_insert}) values ('?',{data_placeholder_insert})'''
                cur = self.conn.cursor()
                cur.execute(sql, values_insert)
                self.conn.commit()
            else:
                print('Makesure that colnames is in python list format `[...,...,...]`')
        else:
            print('Please initialize connection and table name first.')
    
    
class Vibrateserver(Sqlite, BaseHTTPRequestHandler):
    def __init__(self):
        self.root_dir = os.getcwd()
        self.db = 'records.db'
        
        self.db_path = os.path.join(self.root_dir, self.db)

        self.in_sw = {
            'two_corner':['front_facing','side_facing','room_entry'],
            'two_entry':['left_sensor','right_sensor'],
            'one_perimeter':['carport_perimeter','large_gate','small_gate','garden_fence','garden_outer_perimeter'],
            'one_main':['parlor_window','main_door_upper','main_door_lower'],
            'one_hallway':['hallway_door','hallway_window','kitchen_lamp_switch','outdoor_lamp_switch','parlor_lamp_switch']
        }
        self.out_gb = 0
        self.out_sw = {
            'one_hallway':{
                'kitchen_lamp_switch':0,
                'outdoor_lamp_switch':0,
                'parlor_lamp_switch':0
            }
        }
        random.seed(1209218402348023984239874891)
        self.key = self.gethash(random.random())
        self.key = self.gethash(self.key)
        
        self.host = '0.0.0.0'
        self.port = 5000
        
    def do_GET(self):
        try:
            qs = parse_qs(urlparse(self.path).query)
            print(qs)
            path = urlparse(self.path).path
            hwid = qs['hwid'][0]

            if qs['key'][0] == self.key:
                print('key correct')
                print(self.path)
                # Input handler
                if path == f'/input':
                    print('go to input handler')
                    if hwid in self.in_sw.keys():
                        payload_split = qs['payload'].split('z')
                        if len(payload_split) == len(self.in_sw[hwid]):
                            self.set_table(hwid)
                            self.insert_value(self.in_sw[hwid], payload_split)
                        else:
                            self.send_error(500, 'Access denied')
                    else:
                        self.send_error(500, 'Access denied')

                # Output handler
                elif path == f'/output':
                    print('output handler')
                    print(hwid)
                    print(self.out_sw.keys())
                    if hwid in self.out_sw.keys():
                        print('output handler corret hwid')
                        self.send_response(200)
                        self.send_header('Content-type', 'text/html')
                        self.end_headers()
                        self.wfile.write(bytes('0,0,1', 'utf-8'))
                    else:
                        self.send_error(500, 'Access denied')
            else:
                print('full error')
                self.send_error(500, 'Access denied')
           
        except:
            print('exception')
            self.send_error(500, 'Access denied')

In [153]:
vs = Vibrateserver()
vs.key

'431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee'

In [154]:
vs.create_connection(vs.db_path)

In [155]:
type(vs.conn)

sqlite3.Connection

In [156]:
isinstance(vs.conn, sqlite3.Connection)

True

In [94]:
webServer = HTTPServer((vs.host, vs.port), vs)
print(f'Server started http://{vs.host}:{vs.port}')

Server started http://0.0.0.0:5000


In [138]:
vs.host

'0.0.0.0'

In [ ]:
try:
    webServer.serve_forever()
except KeyboardInterrupt:
    pass

In [149]:
vs.path = 'http://localhost:5000/output?key=431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee&hwid=one_hallway'

In [150]:
vs.do_GET()

{'key': ['431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee'], 'hwid': ['one_hallway']}
key correct
http://localhost:5000/output?key=431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee&hwid=one_hallway
output handler
one_hallway
dict_keys(['one_hallway'])
output handler corret hwid


In [130]:
urlparse(vs.path).path

'/output'

In [73]:
from urllib.parse import urlparse, parse_qs
url = 'localhost:8080/input?key=431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee&hwid=two_corner&payload=423z982z912'
qs = parse_qs(urlparse(url).query)
path = urlparse(url).path

In [151]:

qs['hwid'][0] in ag.in_sw.keys()

True

In [80]:
ag.in_sw.keys()

dict_keys(['two_corner', 'two_entry', 'one_perimeter', 'one_main', 'one_hallway'])

In [152]:
'hwid' in

{'key': ['431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee'],
 'hwid': ['two_corner'],
 'payload': ['423z982z912']}

localhost:8080/input?key=431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee&hwid=two_corner&payload=423z982z912

In [85]:
f = [0,0,0]
len(f)

3

In [ ]:
http://192.168.0.5:5000/input?key=431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee&hwid=two_corner&payload=423z982z912

http://192.168.0.5:5000/input?key=431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee&hwid=two_entry&payload=423z982

one_perimeter
http://192.168.0.5:5000/input?key=431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee&hwid=one_perimeter&payload=423z982z998z45z456

In [ ]:
http://192.168.0.5:5000/output?key=431f8a00bb579d4691b19efe332e21d8b61ff25e8a1698d81bf95a756660f3ee&hwid=one_hallway

In [157]:
a = ['front_facing', 'side_facing', 'room_entry']
b = ['423', '982', '912']

In [159]:
all(isinstance(i, list) for i in [a, b])

True

In [160]:
a = ['a', 'b', 'c', 'd']
b = ['float', 'int', 'int', 'int']
for x, y in zip(a, b):
    print(x, y)

a float
b int
c int
d int


In [161]:
a = {
            'two_corner':{
                'reset_trigger':0,
            },
            'two_entry':{
                'reset_trigger':0,
            },
            'one_perimeter':{
                'reset_trigger':0,
            },
            'one_main':{
                'reset_trigger':0,
            },
            'one_hallway':{
                'kitchen_lamp_switch':0,
                'outdoor_lamp_switch':0,
                'parlor_lamp_switch':0,
                'reset_trigger':0
            }
        }

In [172]:
','.join([str(v) for v in a['one_hallway'].values()])

'0,0,0,0'

In [170]:
','.join(a['one_hallway'].values())

TypeError: sequence item 0: expected str instance, int found